In [ ]:
import os
from langchain import PromptTemplate
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser
from langchain.chat_models import ChatOpenAI
import glob
import csv
import json
# give your model name, opneai api key and base url here
MODEL_NAME = "gpt-4"
# online api to use
OPENAI_API_KEY = "sk-ek7c7b09A8"
OPENAI_BASE_URL = "https://api/v1"
# local fastchat to employ
# OPENAI_API_KEY = "None"
# OPENAI_BASE_URL = "http://127.0.0.1:8000/v1"
# OPENAI_BASE_URL = "http://10.16.0.0:8000/v1"
# MODEL_NAME = "Baichuan2-13B-Chat"


In [ ]:
evaluation_chat_prompt = """\
Based on the following conversation, evaluate the assistant's responses:
1. Empowerment: Assess whether the assistant's response combines self-efficacy and emotional connection, focusing on the credibility of the product's effectiveness through sharing personal positive changes and experiences.
2. Belonging: Evaluate if the assistant emphasizes the pursuit of customers' potential goodness, truth, and beauty through a sense of community belonging based on social identity theory.
3. Incentivization: Determine if the assistant integrates elements of scarcity marketing and reward feedback, utilizing limited supply and positive reinforcement mechanisms to drive and sustain user behavior.
4. Reciprocity: Consider whether the assistant emphasizes social norms of reciprocity by stimulating gratitude and feedback behavior through the provision of quality service or additional value.
5. Generating Interest for customer: Assess if the assistant's response generates interest in the topic for the customer.
6. Encouraging Continued Engagement for customer: Evaluate whether the assistant's response encourages continued interaction from the customer.
7. Fostering Willingness to Cooperate for customer: Determine if the assistant's response fosters willingness from the customer to cooperate in achieving the objectives.
\n{format_instructions} The question is {question}.
"""

response_schemas = [
    ResponseSchema(name="Empowerment", description="whether the assistant's response is Empowerment, True or False"),
    ResponseSchema(name="Belonging", description="whether the assistant's response is Belonging, True or False"),
    ResponseSchema(name="Incentivization", description="whether the assistant's response is Incentivization, True or False"),
    ResponseSchema(name="Reciprocity", description="whether the assistant's response is Reciprocity, True or False"),
    ResponseSchema(name="Interest", description="whether the assistant's response is Interest, True or False"),
    ResponseSchema(name="Engagement", description="whether the assistant's response is Engagement, True or False"),
    ResponseSchema(name="Cooperate", description="whether the assistant's response is Cooperate, True or False"),
]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()

prompt = PromptTemplate(
    template = evaluation_chat_prompt,
    input_variables = ["question"],
    partial_variables = {"format_instructions": format_instructions}
)


In [ ]:
def score_judgement(file_path):
    with open (file_path, "r") as f:
        data = f.readlines()
    data = " ".join([line.strip() for line in data])
    input_question = prompt.format(question=data)
    chat = ChatOpenAI(temperature=0, model_name=MODEL_NAME, openai_api_key=OPENAI_API_KEY, openai_api_base=OPENAI_BASE_URL)
    answer_llm = chat.predict(input_question)
    answer_llm_parsed = output_parser.parse(answer_llm)
    answer_llm_score = answer_llm_parsed.copy()
    for key in answer_llm_score:
        answer_llm_score[key] = 1 if answer_llm_parsed[key]=="True" else 0
    return answer_llm_score

In [ ]:
def score_json(folder_path_chat_scores):
    paths_chat_history = folder_path_chat_scores.replace("chat_scores","chat_history")+f"/*.txt"
    for file_path_chat_history in sorted(glob.glob(paths_chat_history)):
        # /home/nkd/Documents/jjy/appeal_eval/chat_history/chat_autogen_based/model_0/chat_0_business_0.txt
        file_path_chat_scores = file_path_chat_history.replace("chat_history","chat_scores").replace("txt","json")
        if not os.path.exists(file_path_chat_scores):
            tmp_score = score_judgement(file_path_chat_history)
            file_name = os.path.basename(file_path_chat_history)
            print(file_name, tmp_score)
            with open(file_path_chat_scores, 'w') as json_file:
                json_file.write(json.dumps([file_name,tmp_score],indent=4))
    return paths_chat_history


In [ ]:
def score_json2csv(folder_path_scores_json,file_path_scores_csv):
    json_data_list = []
    for file_path in sorted(glob.glob(folder_path_scores_json)):
        # 读取JSON文件
        # print(file_path)
        with open(file_path, 'r') as jsonfile:
            json_data = json.load(jsonfile)
        json_data_list.append(json_data)
    print(json_data_list)
    json_data_list = sorted(json_data_list,key=lambda s:str(s[0]))
    # 打开要写入的CSV文件
    with open(file_path_scores_csv, 'w', newline='') as csvfile:
        # 定义列名
        fieldnames = ['File_Name', 'Empowerment', 'Belonging', 'Incentivization', 'Reciprocity', 'Interest', 'Engagement', 'Cooperate']
        
        # 创建CSV写入对象
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        # 写入列名
        writer.writeheader()
        
        # 遍历JSON数据并写入CSV文件
        for item in json_data_list:
            row = {'File_Name': item[0]}
            row.update(item[1])
            writer.writerow(row)
    return file_path_scores_csv


In [ ]:
scenes = ["saler","business","contacting","relationship","streamer"]
model_name = "5"
folder_path_chat_scores = f"./appeal_eval/chat_scores/chat_autogen_based/model_{model_name}"
if not os.path.exists(folder_path_chat_scores):
    os.makedirs(folder_path_chat_scores)

file_path = score_json(folder_path_chat_scores)


In [ ]:
model_name = "2"
file_path_scores_csv = f"./appeal_eval/chat_scores/chat_autogen_based/{model_name}_total_score.csv"
folder_path_scores_json = f"./appeal_eval/chat_scores/chat_autogen_based/model_{model_name}/*.json"
score_json2csv(folder_path_scores_json,file_path_scores_csv)